# Simulated Annealing Implementation

In this file, we have implemented our own version of SA where we have the following random changes to an initial propopsed solution:

1- Randomly change destination of one random station

2- Randomly introduce new station to be visited

3- Randomly remove a station that was being visited
    
Some known issues with the current implementation:

1- SA algorithms should be analyzed by running multiple random starts and observing all results and choosing the best. Since this is a simpler implementation, we have only included one "hard" and one "easy" case as demonstration.

2- Our SA algorithm does not make changes to the first and last visited stations. That means the algorithm could either have moore random changes that include the first and last station specifically, or modify current random changes to include those stations. Another solution would be to run the algorithm multiple times, each with randomly starting and ending stations.

3- The random starts are currently being manually inputed. Ideally, they should be produced randomly.

In [106]:
import random
import numpy as np
import math
random.seed(3102020)
np.random.seed(3102020)

In [322]:
def obj(t,b,N,S):
    rslt = list()
    for i in range(len(b)):
        rslt.append((b[i] - t[i] - N[i]) * S[i] + (N[i] - b[i] +t[i]) * (1-S[i]))
    return sum(rslt)

def organize_trips(X):
    if type(X) != "list":
        X = list(X)
    try:
        pos = [(index, row.index(1)) for index, row in enumerate(X) if 1 in row]
    except:
        pos = [(index, np.argwhere(row==1)) for index, row in enumerate(X) if 1 in row]
    c = list()
    for j in range(len(pos)):
        if j == 0:
            c.append(pos[0])
        else:
            prev_n = c[-1][1]
            bb = list(filter(lambda x: x[0] == prev_n,pos))
            idx = pos.index(bb[0])
            c.append(pos[idx])
    return c

def change_destination(X):
    trips = organize_trips(X)
    rand = np.random.randint(low = 1, high =  len(trips)-1, size =  1)[0]
    prev_trips = trips[0:rand]
    prohibited = list()
    for station in prev_trips:
        prohibited.append(station[1])
    m = len(X)
    destination = random.choice([c for c in range(m) if c not in [prohibited]])
    provenient = trips[rand][0]
    trips[rand] = (provenient,destination)
    rm_stations = trips[rand+1:len(trips)]
    trips = [x for x in trips if x not in rm_stations]
    for j in range(len(rm_stations)):
        for i in range(len(rm_stations)):
            if rm_stations[i][0] == trips[-1][1]:
                trips.append(rm_stations[i])
    last_trip_start = trips[-1][0]
    trips[-1] = (last_trip_start,0)    
    return trips

def add_station(Xbest):
    no_trips = [0 for i in range(len(Xbest))]
    j = 0
    for i in Xbest:
        j = j + 1
        if sum(i) == 0:
            no_trips[j-1] = 1
    stat_no_trips = list()
    n = 0
    for i in no_trips:
        if i == 1:
            stat_no_trips.append(n)
        n += 1
    if len(stat_no_trips) == 0:
        return organize_trips(Xbest)
    new_destination = random.choice(stat_no_trips)
    trips_org = organize_trips(Xbest)
    trip_numbs = [i for i in range(len(trips_org))]
    trip_numbs.pop(len(trip_numbs)-1)
    trip_numbs.pop(0)
    trip_insert = random.choice(trip_numbs)
    trips_before = trips_org[0:trip_insert]
    trips_after = trips_org[trip_insert:len(trips_org)]

    prior_station = trips_before[-1][0]
    posterior_station = trips_after[0][0]
    new_trip = [(new_destination, posterior_station)]
    trips_before[-1] = (prior_station, new_destination)
    trips = trips_before + new_trip + trips_after
    return trips

def rm_station(Xbest):
    trips_org = organize_trips(Xbest)
    trip_numbs = [i for i in range(len(trips_org))]
    trip_numbs.pop(len(trip_numbs)-1)
    trip_numbs.pop(0)
    trip_rm = random.choice(trip_numbs)
    trips_before = trips_org[0:trip_rm]
    trips_after = trips_org[trip_rm:len(trips_org)]
    prior_station = trips_before[-1][0]
    posterior_station = trips_after[0][0]
    trips_before[-1] = (prior_station, posterior_station)
    trips = trips_before + trips_after
    return trips

def get_new_x(x,trips):
    s = len(x)
    new_x = np.zeros((s,s))
    cd = trips
    for i in range(len(cd)):
        new_x[cd[i]] = 1
    return new_x

def check_time(times,new_x,T):
    sum_list = list()
    for i in range(len(times)):
        for j in range(len(times)):
            sum_list.append(times[i][j]*new_x[i,j])
        return sum(sum_list) > T
    
def get_new_tr(trips,c_max,b,N,x):
    c = 0
    ds = [b[i] - N[i] for i in range(len(N))]
    bikes_change = list()
    for i in range(len(trips)):
        if i == 0:
            bikes_change.append(0)
        else:
            s = trips[i][0]
            c_new = c_max if c + ds[s-1] > c_max else max(0,c + ds[s-1])
            bikes_change.append(c_new - c)
            c = c_new
    tr_new = list()
    for i in range(len(x)):
        tr_new.append(0)
    for i in range(len(trips)):
        if i == 0:
            continue
        else:
            s = trips[i][0]
            stations_order = [t[0] for t in trips]
            station_trip_idx = stations_order.index(s)
            tr_new[s-1] = bikes_change[station_trip_idx]
    return tr_new

def SA(X,TR,tp,K,alpha,imax, ni_max,times,max_T = 60, c_max = 15,b,N,S):
    i = 0
    ni = 0
    Xstart = X
    X_old = X
    Xbest = X
    TRbest = TR
    TR_old = TR
    BEST = obj(TR,b,N,S)
    while ni < ni_max:
        Xbest = list(Xbest)
        P = random.uniform(0,1)
        if P < 0.3: 
            trips = change_destination(X_old)
            X = get_new_x(Xstart,trips)
            if check_time(times,X,max_T):
                continue
            TR_new = get_new_tr(trips,c_max,b,N,X)
        elif P < 0.6:
            trips = add_station(X_old)
            X = get_new_x(Xstart,trips)
            if check_time(times,X,max_T):
                continue
            TR_new = get_new_tr(trips,c_max,b,N,X)
        else:
            trips = rm_station(X_old)
            X = get_new_x(Xstart,trips)
            if check_time(times,X,max_T):
                continue
            TR_new = get_new_tr(trips,c_max,b,N,X)
        i += 1
        diff = obj(TR_new,b,N,S) - obj(TR_old,b,N,S)
        if diff <= 0:
            TR_old = TR_new
            X_old = X
        else:
            r = random.uniform(0,1)
            if r > math.exp(diff/(K * tp)):
                TR_old = TR_new
            else:
                ni = ni + 1
                continue
        if obj(TR_old,b,N,S) < BEST:
            Xbest = X_old
            TRbest = TR_old
            BEST = obj(TRbest,b,N,S)
        if i == imax:
            tp = alpha * tp
            i = 0
    return {"objective" : BEST, "X" : Xbest, "TR" : TRbest}


## Examples

The following are examples for the implementation of the SA algorithm. They include 19 stations randomly selected to have their bikes optimized. We identified one "hard" case and one "easy" case, the former which took hours to complete while the easy case took only a few seconds.

The initial X was randomly and manually assigned, while b, N and S were given as a result of our demand and supply efforts in other works.

The results include a print of every successful iteration for a "Best" case; the final results with objective value, X array that shows each trip and "TR" which means the bikes taken / received at each station.

In [325]:
import pandas as pd
tss = pd.read_csv("~/dist_matrix_0.csv")
times = np.array(tss)

In [324]:
# "Hard" case
b = [5,9,15,18,11,15,18,15,22,11,10,16,10,6,8,10,23,15,8,16,8,5,12,11,11,17,13,11]
N = [9,23,4,9,10,11,21,10,10,8,6,9,9,15,15,11,9,23,15,9,4,7,8,13,9,9,10,6]
S = [1 if N[i] < b[i] else 0 for i in range(len(b))]
X = [[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
TR = [0,-14,11,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
tp = 1000000000000000 # initial temperature
alpha = 0.9 # temperature decreasing rate
K = 0.00000001 # temperature-related constant to test against random value
imax = 10 # Tries before lowering temperature
ni_max = 3000 # Failure to improve before ending loop 

rslt = SA(X,TR,tp,K,alpha,imax, ni_max,times,max_T = 60,c_max = 20,b,N,S)

rslt

122
118
114
113
109
99
95
89
75
69
65
60
48
44
41
37
35
31


{'objective': 31,
 'X': [array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]),
  array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [320]:
# "Easy" case
b = [5,12,12,4,7,5,10,12,4,7,10,8,8,9,6,6,4,5,6,7,6,7,9,9,10,9,6,8]
N = [6,8,5,10,9,6,8,13,8,14,7,4,9,5,3,4,6,13,8,4,9,3,7,6,6,8,9,6]
S = [1 if N[i] < b[i] else 0 for i in range(len(b))]
X = [[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
TR = [0,0,7,-6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0]
tp = 1000000000000000 # initial temperature
alpha = 0.9 # temperature decreasing rate
K = 0.000000000001 # temperature-related constant to test against random value
imax = 10 # Tries before lowering temperature
ni_max = 3000 # Failure to improve before ending loop 

rslt = SA(X,TR,tp,K,alpha,imax, ni_max,times,max_T = 60,c_max = 20)

rslt

69
66
61
60
56
52
46
40
38
34
26
22
19
17
13
11
10


{'objective': 10,
 'X': [array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
  array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 